# CSCI 3202, Spring 2022: Final Coding Project

---


This is your final programming project for CSCI 3202. It is due on Canvas by **11:59 PM on Saturday April 30**.  Your solutions to theoretical questions should be done in Markdown/LateX directly below the associated question. Your solutions to computational questions should include any relevant Python code, as well as results and any written commentary.

You have two options for completing your final project for this course. The first option is presented in this notebook and involves implementing a reinforcement learning algorithm and producing a five-minute video that explains your process of solving this problem. The second option is to design your own project that includes the algorithms we've discussed since the midterm - Bayes Nets, Hidden Markov Models, Markov Decision Processes, or Reinforcement Learning - or an algorithm related to one of these that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. You will also need to produce a short, five-minute video that explains your project.

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- The project needs to be approved by the course instructors.
- The project needs to include one of the algorithms we've discussed since the midterm - Bayes Nets, HMMs, MDPs, or Reinforcement Learning - or an algorithm that we haven't discussed in class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of your final exam. So, we will be much less helpful than we typically am with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If something is left open-ended, it is probably because we intend for you to code it up however you want, and only care about the plots/analysis we see at the end. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

Glenn Holzhauer

---



In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
import random

from datetime import datetime

# added packages
import heapq
from matplotlib import colors



---
## [100 pts] Problem 1:  Reinforcement learning

Consider a **cube** state space defined by $0 \le x, y, z \le L$. Suppose you are piloting/programming a drone to learn how to land on a platform at the center of the $z=0$ surface (the bottom). Some assumptions:
* In this discrete world, if the drone is at $(x,y,z)$ it means that the box is centered at $(x,y,z)$. There are boxes (states) centered at $(x,y,z)$ for all $0 \le x,y,z \le L$. Each state is a 1 unit cube. So when $L=2$ (for example), there are cubes centered at each $x=0,1,2$, $y=0,1,2$ and so on, for a total state space size of $3^3 = 27$ states.
* All of the states with $z=0$ are terminal states.
* The state at the center of the bottom of the cubic state space is the landing pad. For example, when $L=4$, the landing pad is at $(x,y,z) = (2,2,0)$.
* All terminal states ***except*** the landing pad have a reward of -1. The landing pad has a reward of +1.
* All non-terminal states have a living reward of -0.01.
* The drone takes up exactly 1 cubic unit, and begins in a random non-terminal state.
* The available actions in non-terminal states include moving exactly 1 unit Up (+z), Down (-z), North (+y), South (-y), East (+x) or West (-x). In a terminal state, the training episode should end.

#### Part A
Write a class `MDPLanding` to represent the Markov decision process for this drone. Include methods for:
1. `actions(state)`, which should return a list of all actions available from the given state
2. `reward(state)`, which should return the reward for the given state
3. `result(state, action)`, which should return the resulting state of doing the given action in the given state

and attributes for:
1. `states`, a list of all the states in the state space, where each state is represented as an $(x,y,z)$ tuple
2. `terminal_states`, a dictionary where keys are the terminal state tuples and the values are the rewards associated with those terminal states
3. `default_reward`, a scalar for the reward associated with non-terminal states
4. `all_actions`, a list of all possible actions (Up, Down, North, South, East, West)
5. `discount`, the discount factor (use $discount = 0.999$ for this entire problem)

How you feed arguments/information into the class constructor is up to you.

Note that actions are *deterministic* here.  The drone does not need to learn transition probabilities for outcomes of particular actions. What the drone does need to learn, however, is where the landing pad is, and how to get there from any initial state.

Before moving on to Part B, we recommend that you test that your MDPLanding code is set up correctly. Write unit tests that display the actions for a given state, rewards, results, etc. This will help you identify errors in your implementation and save you a lot of debugging time later.

In [2]:
# Your code here.
random.seed(datetime.now())
gamma = .999



class MDPLanding:
    def __init__(self, L, terminallist, defreward, gamma):


        self.terminals = terminallist
        self.defrewards = defreward
        self.gamma = gamma
        self.L = L
        self.states = [(x, y, z) for x in range(L) for y in range(L) for z in range(L)]

    def actions(self, state):
        actlist = []
        if state in self.terminals:
            return []#no valid moves in terminal state
        else:
            if state[0] - 1 >= 0:
                actlist.append('-x')

            if state[0] + 1 < self.L: 
                actlist.append('+x')

            if state[1] - 1 >= 0: 
                actlist.append('-y')

 
            if state[1] + 1 < self.L:
                actlist.append('+y')

            if state[2] - 1 >= 0: 
                actlist.append('-z')

            if state[2] + 1 < self.L:
                actlist.append("+z")

        return actlist
            
        
    def reward(self, state):
        if state in self.terminals: 
            return self.terminals[state]
        else: 

            return self.defrewards
 
        
    def result(self, state, dir):
        
        if dir == '+x':
            return (state[0] +1, state[1], state[2])
        elif dir == '-x':
            return (state[0] - 1, state[1], state[2])
        elif dir == '+y':
            return (state[0], state[1]+ 1, state[2])
        elif dir == '-y':
            return (state[0], state[1]- 1, state[2])
        elif dir == '+z':
            return (state[0], state[1], state[2] + 1)
        elif dir == '-z':
            return (state[0], state[1], state[2] -1)
        elif dir == [None] or dir == None:
            return (state)
        
        

#### Part B
Write a function to implement **policy iteration** for this drone landing MDP. Create an MDP statesironment to represent the $L=4$ case (so 125 total states).

Use your function to find an optimal policy for your new MDP environment. Check (by printing to screen) that the policy for the following states are what you expect, and **comment on the results**:
1. $(2,2,1)$
1. $(0,2,1)$
1. $(2,0,1)$

The policy for each of these states is the action that the agent should take in that state. 

In [3]:
# Your code here.





def utility(mdp, action, s, U):
    u = 0

    u = mdp.gamma*U[mdp.result(s, action)] + mdp.reward(mdp.result(s, action))


    return u


def policy_iteration(mdp, pi, U):
    notchanged = False
    while not notchanged:
        notchanged = True
        for state in mdp.states:
            best_util = utility(mdp, pi[state], state, U)
            best_act = pi[state]
            U[state] = best_util

            for action in mdp.actions(state):

                attempt = utility(mdp, action, state, U)
                if attempt > best_util and action != pi[state]:
                    best_act = action
                    U[state] = attempt
                    notchanged = False

            pi[state] = best_act

    return pi



L = 5 #realized late in design that my L is the normal L minus 1 - need to compensate by increasing L by one
grid =[(x, y, z) for x in range(L) for y in range(L) for z in range(L)] #basis list for all dictionaries using positions

pi = dict.fromkeys(grid, None) #unintelligent policy




tstates = [(x, y, z) for x in range(L) for y in range(L) for z in range(1)] ##terminal states are being generated

land = (2, 2, 0) #location for landing
terminal = dict.fromkeys(tuple(tstates), -1) #generating terminal rewards 

terminal[land] = 1 #generating terminal correct landing location

U = dict.fromkeys(grid, 0) #utility "vector"


default_reward = -0.01
gamma = 0.999
mdp = MDPLanding(5, terminal, default_reward, gamma)
s = (2, 2, 1)
s2 = (0, 2, 1)
s3 = (2, 0, 1)


newpi = policy_iteration(mdp, pi, U)
print(newpi[s]) #for 2,2,1 - this is expected
print(newpi[s2])
print(newpi[s3])


print(newpi)




-z
+x
+y
{(0, 0, 0): None, (0, 0, 1): '+y', (0, 0, 2): '-z', (0, 0, 3): '-z', (0, 0, 4): '-z', (0, 1, 0): None, (0, 1, 1): '+y', (0, 1, 2): '-z', (0, 1, 3): '-z', (0, 1, 4): '-z', (0, 2, 0): None, (0, 2, 1): '+x', (0, 2, 2): '-z', (0, 2, 3): '-z', (0, 2, 4): '-z', (0, 3, 0): None, (0, 3, 1): '-y', (0, 3, 2): '-z', (0, 3, 3): '-z', (0, 3, 4): '-z', (0, 4, 0): None, (0, 4, 1): '-y', (0, 4, 2): '-z', (0, 4, 3): '-z', (0, 4, 4): '-z', (1, 0, 0): None, (1, 0, 1): '+y', (1, 0, 2): '-z', (1, 0, 3): '-z', (1, 0, 4): '-z', (1, 1, 0): None, (1, 1, 1): '+y', (1, 1, 2): '-z', (1, 1, 3): '-z', (1, 1, 4): '-z', (1, 2, 0): None, (1, 2, 1): '+x', (1, 2, 2): '-z', (1, 2, 3): '-z', (1, 2, 4): '-z', (1, 3, 0): None, (1, 3, 1): '-y', (1, 3, 2): '-z', (1, 3, 3): '-z', (1, 3, 4): '-z', (1, 4, 0): None, (1, 4, 1): '-y', (1, 4, 2): '-z', (1, 4, 3): '-z', (1, 4, 4): '-z', (2, 0, 0): None, (2, 0, 1): '+y', (2, 0, 2): '-z', (2, 0, 3): '-z', (2, 0, 4): '-z', (2, 1, 0): None, (2, 1, 1): '+y', (2, 1, 2): '-z', (2, 

#### Part C

Code up a **Q-learning** agent/algorithm to learn how to land the drone. You can do this however you like, as long as you use the MDP class structure defined above.  

Your code should include some kind of a wrapper to run many trials to train the agent and learn the Q values.  You also do not need to have a separate function for the actual "agent"; your code can just be a "for" loop within which you are refining your estimate of the Q values.

From each training trial, save the cumulative gammaed reward (utility) over the course of that episode. That is, add up all of $\gamma^t R(s_t)$ where the drone is in state $s_t$ during time step $t$, for the entire sequence. We refer to this as "cumulative reward" because we usually refer to "utility" as the utility *under an optimal policy*.

Some guidelines:
* The drone should initialize in a random non-terminal state for each new training episode.
* The training episodes should be limited to 50 time steps, even if the drone has not yet landed. If the drone lands (in a terminal state), the training episode is over.
* You may use whatever learning rate $\alpha$ you decide is appropriate, and gives good results.
* There are many forms of Q-learning. You can use whatever you would like, subject to the reliability targets in Part D below.
* Your code should return:
  * The learned Q values associated with each state-action pair.
  * The cumulative reward for each training trial. 
  * Anything else that might be useful in the ensuing analysis.

In [4]:
# Your code here.
alpha = .1
gamma = .999



L = 11
grid2 =[(x, y, z) for x in range(L) for y in range(L) for z in range(L)]

tstates2 = [(x, y, z) for x in range(L) for y in range(L) for z in range(1)] ##terminal states are being generated
terminal2 = dict.fromkeys(tuple(tstates2), -1) #generating terminal rewards 
land = (2, 2, 0) #location for landing
terminal2[land] = 1
mdpQ = MDPLanding(11, terminal2, -.01, gamma)
tupls = [('-x', 0), ('+x', 0), ("-y", 0), ("+y", 0),("-z", 0),("+z", 0)]

Q = dict.fromkeys(grid2, tupls)

pi2 = dict.fromkeys(grid2, None)

chance_for_exploration = 1
minexp = .01

totals = []
episodes = 5
maxsteps = 50


for i in range(episodes):
    rewardtotal = 0
    terminated = False
    x = random.randint(1, 10)
    y = random.randint(1, 10)
    z = random.randint(1, 10)
    s = (x, y, z)


    for j in range(maxsteps):
        if(np.random.uniform(0, 1) < chance_for_exploration):
            act = random.choice(mdpQ.actions(s))

        else:
            valid_action_not_found = True
            while(valid_action_not_found):
                bestind = 0

                for k in range(6):
                    if Q[s][k][1] > Q[s][bestind][1]:
                        bestind = k
                act = Q[s][bestind][0]
                if act in mdpQ.actions(s):
                    valid_action_not_found = False

        s_next = mdpQ.result(s, act)

        r = mdpQ.reward(s_next)

        if(mdpQ.actions(s_next) == []):
            terminated = True

        actind = 0

        for l in range(6):
            if(Q[s][l][0] == act):
                bestind2 = 0
                for k in range(6):
                    if Q[s_next][k][1] > Q[s_next][bestind2][1]:
                        bestind2 = k
                oftup = list(Q[s][bestind2])
                
     
                oftup[1] = (1-alpha) * Q[s][l][1] + alpha*(r + gamma*Q[s_next][bestind2][1])
                Q[s][l] = oftup
        rewardtotal = rewardtotal+ r
        if(terminated):
            break
        s = s_next
    chance_for_exploration = chance_for_exploration-.01


    totals.append(rewardtotal)

print(totals[2])



-0.5000000000000002


#### Part D

Initialize the $L=10$ environment (so that the landing pad is at $(5,5,0)$). Run some number of training trials to train the drone.

**How do I know if my drone is learned enough?**  If you take the mean cumulative reward across the last 5000 training trials, it should be around 0.80. This means at least about 10,000 (but probably more) training episodes will be necessary. It will take a few seconds on your computer, so start small to test your code.

**Then:** Compute block means of cumulative reward from all of your training trials. Use blocks of 500 training trials. This means you need to create some kind of array-like structure such that its first element is the mean of the first 500 trials' cumulative rewards; its second element is the mean of the 501-1000th trials' cumulative rewards; and so on. Make a plot of the block mean rewards as the training progresses. It should increase from about -0.5 initially to somewhere around +0.8.

**And:** Print to the screen the mean of the last 5000 trials' cumulative rewards, to verify that it is indeed about 0.80.

In [5]:
# Your code here.
alpha = .1
gamma = .999



L = 11
grid3 =[(x, y, z) for x in range(L) for y in range(L) for z in range(L)]


tstates3 = [(x, y, z) for x in range(L) for y in range(L) for z in range(1)] ##terminal states are being generated
terminal3 = dict.fromkeys(tuple(tstates3), -1) #generating terminal rewards 
land = (2, 2, 0) #location for landing
terminal3[land] = 1
mdpQ2 = MDPLanding(11, terminal3, -.01, gamma)


tupls = [('-x', 0), ('+x', 0), ("-y", 0), ("+y", 0),("-z", 0),("+z", 0)]

Q = dict.fromkeys(grid3, tupls)

pi2 = dict.fromkeys(grid3, None)

chance_for_exploration = 1
minexp = .01

totals = []
episodes = 10000
maxsteps = 50
for i in range(episodes):
    rewardtotal = 0
    terminated = False
    x = random.randint(1, 10)
    y = random.randint(1, 10)
    z = random.randint(1, 10)
    s = (x, y, z)


    for j in range(maxsteps):
        if(np.random.uniform(0, 1) < chance_for_exploration):
            act = random.choice(mdpQ.actions(s))

        else:
            valid_action_not_found = True
            while(valid_action_not_found):
                bestind = 0

                for k in range(6):
                    if Q[s][k][1] > Q[s][bestind][1]:
                        bestind = k
                act = Q[s][bestind][0]
                if act in mdpQ.actions(s):
                    valid_action_not_found = False

        s_next = mdpQ.result(s, act)

        r = mdpQ.reward(s_next)

        if(mdpQ.actions(s_next) == []):
            terminated = True

        actind = 0

        for l in range(6):
            if(Q[s][l][0] == act):
                bestind2 = 0
                for k in range(6):
                    if Q[s_next][k][1] > Q[s_next][bestind2][1]:
                        bestind2 = k
                oftup = list(Q[s][bestind2])
                
     
                oftup[1] = (1-alpha) * Q[s][l][1] + alpha*(r + gamma*Q[s_next][bestind2][1])
                Q[s][l] = oftup
        rewardtotal = rewardtotal+ r
        if(terminated):
            break
        s = s_next
    chance_for_exploration = chance_for_exploration-.01


    totals.append(rewardtotal)

print(totals[2])




mid = 5000
firstfive = 500



first_block = totals[:firstfive] #if this did work in computing time, this would be the block of first 500 trials
first_mean = sum(first_block)/500



second_block = totals[firstfive:]

second_mean = sum(second_block)/9500 #and this would be the second block for the final 9500





third_block = totals[mid:]

third_mean = sum(third_block)/5000 #for second block of 5000 episodes

print(third_mean)











KeyboardInterrupt: 

#### Part E

**Question 1:** Why does the cumulative reward start off around -0.5 at the beginning of the training?

The drone has not yet learned how to land in the desired terminal states, which means that it will most often be receiving rewards of -1 or flying around and receiving the default reward of -.01, neither of which will give the drone a cumulatively positive reward. This is basically the equivalent of saying the drone isn't properly trained so it is not completing the task of landing in the desired position well. This cumulative value will increase over time as the drone receives positive +1 reward instead.

**Question 2:** Why will it be difficult for us to train the drone to reliably obtain rewards much greater than about 0.8?

The rewards are still going to steadily decline with default rewards being negative, and the maximum cumulative reward will be less than 1. There is also randomness to pick a different path rarely that will yield negative rewards from terminals (though this becomes incredibly unlikely)

#### Part F
Choose three other reward structures, including the terminal rewards and the living rewards, and rerun your policy iteration and Q-learning algorithms with those reward structures. Write a paragraph or two describing the reward structures that you selected, what impact you expected the changes to have on your results, and what changes actually happened. If there was a difference in what you expected and what actually happened, reflect on why there was a difference. 



Reward structure 1:
Changing living rewards to 0 and leaving the terminal rewards



In this reward structure, there would be very little weight given to getting to the destination the fastest; the Q-Learning algorithm would not only place weight in getting to the destination itself. The policy may determine a roundabout way to get to the desired landing pad, but will still be trained to find the landing pad. This would probably be very inefficient because randomness can affect results and yield paths that were generated from early-on randomly made decisions.



Reward structure 2: 
Changing living rewards to -.03 and leaving terminal rewards



In this reward structure, the amount of moves to get to the objective would be weighted more heavily, but ultimately this reward structure should not affect the outcome of policy decisions. Even if the living rewards are more negative, the outcome should be the same, because the same best utility paths will arise; actions are only being weighed against eachother, and their negative weights are all equivalent in living rewards. I don't anticipate this policy will look any different when generated. I picked this structure to see if the policy might be able to make the drone move more directly to the designated location.



Reward structure 3:
Leave living rewards the same and adjust terminal rewards for wrong landings to be -5

In this reward structure, the wrong landings are much worse than before; this will most likely end in the agent avoiding very low Z axis regions. I picked this structure because I would want to test out the merits of making errors much greater weights. I was mostly curious to see if x and y movements would be made more immediately in an effort to avoid the regions that are above the landing pad; I think they would. 



REFLECTION :

In each case with the Policy_iteration, the results were actually all identical.  When approaching the z = 1 plane, the drone flies then towards the reward pad with +x and +y. As it turns out, it seems effectively all the reward structure adjustments I made created a very similar situation, in the case of policy iteration. I anticipate that this would not be the case with the QLearning algorithm; because it does not completely deterministically resolve the policy for every state, the pathing would likely be very different to the end goal. With only the training data it creates itself instead of a full policy_iteration pass, these results would have most likely looked very different because the QLearning would have not been so effective in immediately finding the proper actions for each state - there is randomness involved in it's decision making to split from the most likely path.


In [6]:
grid3 =[(x, y, z) for x in range(L) for y in range(L) for z in range(L)]

mdpQs1 = MDPLanding(11, terminal3, 0, gamma)

U2 = dict.fromkeys(grid3, 0) #utility "vector"

pi2 = dict.fromkeys(grid3, None) #unintelligent policy

policy_iteration(mdpQs1, pi2, U2)


mdpQs2 = MDPLanding(11, terminal3, -.03, gamma)

U3 = dict.fromkeys(grid3, 0) #utility "vector"

pi3 = dict.fromkeys(grid3, None) #unintelligent policy

policy_iteration(mdpQs1, pi3, U3)




tstates4 = [(x, y, z) for x in range(L) for y in range(L) for z in range(1)] ##terminal states are being generated
terminal4 = dict.fromkeys(tuple(tstates4), -5) #generating terminal rewards 
land2 = (5, 5, 0) #location for landing
terminal4[land2] = 1



mdpQs3 = MDPLanding(11, terminal4, -.01, gamma)

U4 = dict.fromkeys(grid3, 0) #utility "vector"

pi4 = dict.fromkeys(grid3, None) #unintelligent policy


policy_iteration(mdpQs1, pi4, U4)






{(0, 0, 0): None,
 (0, 0, 1): '+y',
 (0, 0, 2): '-z',
 (0, 0, 3): '-z',
 (0, 0, 4): '-z',
 (0, 0, 5): '-z',
 (0, 0, 6): '-z',
 (0, 0, 7): '-z',
 (0, 0, 8): '-z',
 (0, 0, 9): '-z',
 (0, 0, 10): '-z',
 (0, 1, 0): None,
 (0, 1, 1): '+y',
 (0, 1, 2): '-z',
 (0, 1, 3): '-z',
 (0, 1, 4): '-z',
 (0, 1, 5): '-z',
 (0, 1, 6): '-z',
 (0, 1, 7): '-z',
 (0, 1, 8): '-z',
 (0, 1, 9): '-z',
 (0, 1, 10): '-z',
 (0, 2, 0): None,
 (0, 2, 1): '+x',
 (0, 2, 2): '-z',
 (0, 2, 3): '-z',
 (0, 2, 4): '-z',
 (0, 2, 5): '-z',
 (0, 2, 6): '-z',
 (0, 2, 7): '-z',
 (0, 2, 8): '-z',
 (0, 2, 9): '-z',
 (0, 2, 10): '-z',
 (0, 3, 0): None,
 (0, 3, 1): '-y',
 (0, 3, 2): '-z',
 (0, 3, 3): '-z',
 (0, 3, 4): '-z',
 (0, 3, 5): '-z',
 (0, 3, 6): '-z',
 (0, 3, 7): '-z',
 (0, 3, 8): '-z',
 (0, 3, 9): '-z',
 (0, 3, 10): '-z',
 (0, 4, 0): None,
 (0, 4, 1): '-y',
 (0, 4, 2): '-z',
 (0, 4, 3): '-z',
 (0, 4, 4): '-z',
 (0, 4, 5): '-z',
 (0, 4, 6): '-z',
 (0, 4, 7): '-z',
 (0, 4, 8): '-z',
 (0, 4, 9): '-z',
 (0, 4, 10): '-z',
 (0, 